In [31]:
import re

In [32]:
class STATUS:
    time        :int    # sec
    util        :int    # %
    seg_valid   :int   # # of segments
    seg_dirty   :int
    seg_prefree :int
    seg_free    :int
    cp_calls    :int    # counts
    cp_fg_calls :int    # counts
    cp_bg_calls :int    # counts
    gc_calls    :int    # counts
    memory      :int  # kB
    def parseInt(self):
        self.time = int(self.time)
        self.util = int(self.util)
        self.seg_valid = int(self.seg_valid)
        self.seg_dirty = int(self.seg_dirty)
        self.seg_prefree = int(self.seg_prefree)
        self.cp_calls = int(self.cp_calls)
        self.cp_bg_calls = int(self.cp_bg_calls)
        self.cp_fg_calls = self.cp_calls - self.cp_bg_calls
        self.gc_calls = int(self.gc_calls)
        self.memory = int(self.memory)


In [33]:
def parseStatus(status:str):
    raw = STATUS()
    try:
        idx = 0
            # Header
        raw.time = re.findall(r'Current Time Sec:\s([\d]+)\s\/', status)[0]
        raw.util = re.findall(r'Utilization:\s([\d]+)\%', status)[0]
        
        raw.seg_valid = re.findall(r'Valid:\s([\d]+)', status)[0]
        raw.seg_dirty = re.findall(r'Dirty:\s([\d]+)', status)[0]
        raw.seg_prefree = re.findall(r'Prefree:\s([\d]+)', status)[0]
        raw.seg_free = re.findall(r'Free:\s([\d]+)', status)[0]
        raw.cp_calls = re.findall(r'CP calls:\s([\d]+)\s', status)[0]
        raw.cp_bg_calls = re.findall(r'CP calls:\s[\d]+\s\(BG:\s([\d]+)\)', status)[0]
        raw.gc_calls = re.findall(r'GC calls:\s([\d]+)\s', status)[0]
        raw.memory = re.findall(r'Memory:\s([\d]+)\sKB', status)[0]

        return raw
    except:
        print(status)
        print('except: ', raw.__dict__)

In [34]:
def parseTrace(filename):
    infile = open(filename, 'r')

    idx = 0
    lines=[]
    result=[]
    for line in infile.readlines():
        if not line : continue
        # print(idx, line)
        lines.append(line)
        if('paged :' in line):
            # if(len(lines) > 1):
            result.append(parseStatus(' '.join(lines)))
            lines.clear()
        
        idx += 1
    
    # result.append(parseStatus(' '.join(lines))) # for remain lines
    
    infile.close()
    outfile = open(filename + '.parse', 'w')
    outfile.write(','.join(result[0].__dict__.keys()) + '\n')
    for item in result:
        # outfile.write(item.)
        outfile.write(','.join(item.__dict__.values()) + '\n')
    
    outfile.close()

    return outfile.name

In [35]:
filename = './raw/f2fs_status.txt'
parsedFile = parseTrace(filename)